<a href="https://colab.research.google.com/github/srikacharla/CSC8980/blob/main/Homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Sri Harsha Kacharla
Student ID: 002556509

1.Take the positive and the negative tweets only. Use Sklearn to split the dataset in 80%
training, 20% testing splits. Provide a nicely formatted summary of these splits,
containing their size) (15 points)

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
!mkdir -p data
!wget -nc http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip -P data
!unzip /content/data/trainingandtestdata.zip

URL transformed to HTTPS due to an HSTS policy
File ‘data/trainingandtestdata.zip’ already there; not retrieving.

Archive:  /content/data/trainingandtestdata.zip
replace testdata.manual.2009.06.14.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
import pandas as pd
import numpy as np

training_data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', names=['polarity', 'id', 'date', 'query', 'name', 'text'])
testing_data = pd.read_csv('testdata.manual.2009.06.14.csv', names=['polarity', 'id', 'date', 'query', 'name', 'text'])

In [4]:
training_data.query('polarity == "0" or polarity == "4"')
np.random.seed(2361)
x_train, x_test, y_train, y_test = train_test_split(training_data.drop('polarity', axis='columns'), training_data.polarity,train_size=0.80)

In [5]:
y_train = y_train.to_frame()
y_test = y_test.to_frame()
split_size = [x_train.size, x_test.size, y_train.size, y_test.size]

In [6]:
df = pd.DataFrame({'size of data':split_size}, index = ['x training dataset size', 'x testing dataset size', 'y training dataset size', 'y testing dataset size'])
df

,size of data
x training dataset size,6400000
x testing dataset size,1600000
y training dataset size,1280000
y testing dataset size,320000


2. Use the code from the previous classes to build the following models (15 points):

  a) SVM using TF-IDF.

  b) Naive Bayes using TF-IDF.

  c) Random Forest using TF-IDF.

In [8]:
import sklearn
np.random.seed(2361)
model_nb = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_nb.fit(x_train.text.values.ravel(), y_train.values.ravel())
labels2 = model_nb.predict(x_test.text.values)

In [9]:
np.random.seed(2361)
y_train_values = y_train.values.ravel()
y_train_values = y_train_values.astype(np.int8)
model_rf = make_pipeline(TfidfVectorizer(), RandomForestClassifier(n_jobs=-1,warm_start=True,max_depth=1000))
model_rf.fit(x_train.text.values.ravel(), y_train_values)
labels3 = model_rf.predict(x_test.text.values)


In [10]:
np.random.seed(2361)
model_svm = make_pipeline(TfidfVectorizer(), SVC(max_iter=20000))
model_svm.fit(x_train.text.values.ravel(), y_train_values)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape=

3. Use the code from the LSTM class to build a classifier for negative and positive
sentiment tweets. Train the model with the training data split. Once the model is built,
test it with the testing data split. Display the classifier report for this evaluation. Answer
the following question: What can you say about the performance of this model? (40
points)

In [11]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=280)
tokenizer.fit_on_texts(training_data.text.values) 
sequences_train = tokenizer.texts_to_sequences(x_train.text.values)
sequences_test = tokenizer.texts_to_sequences(x_test.text.values)

In [12]:
!pip install numpy==1.16.2

import tensorflow as tf 
import numpy as np
from tensorflow.keras.preprocessing import sequence
from numpy import array
from sklearn.metrics import classification_report


review_length = 280
vocab_size = 10000
x1_train = sequence.pad_sequences(sequences_train, maxlen = review_length)
x1_test = sequence.pad_sequences(sequences_test, maxlen = review_length)
model = tf.keras.models.Sequential()

model.add(
    tf.keras.layers.Embedding(
        input_dim = vocab_size, 
        output_dim = 32, 
        input_length = review_length 
    )
)
model.add(
    tf.keras.layers.Dropout(
        rate=0.25 
    )
)
model.add(
    tf.keras.layers.LSTM(
        units=32 
    )
)
model.add(
    tf.keras.layers.Dropout(
        rate=0.25 
    )
)
model.add(
    tf.keras.layers.Dense(
        units=1,
        activation='sigmoid'
    )
)
model.compile(
    loss=tf.keras.losses.binary_crossentropy, 
    optimizer=tf.keras.optimizers.Adam(), 
    metrics=['accuracy']) 
model.summary()
y_train1 = y_train
y_test1 = y_test
y_train1 = y_train1.replace(4,1)
y_test1 = y_test1.replace(4,1)
history = model.fit(x1_train, y_train1,batch_size=256,epochs=3,validation_split=0.2,verbose=1)

predicted_classes = model.predict_classes(x1_test)
print(predicted_classes)
classification_report = classification_report(y_test1, predicted_classes)
print(classification_report)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 280, 32)           320000    
_________________________________________________________________
dropout (Dropout)            (None, 280, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
125/125 [==============================] - 7s 29ms/step - loss: 0.6739 - accuracy: 0.5791 - val_loss: 0.56

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[0]
 [1]
 [1]
 ...
 [0]
 [1]
 [1]]
              precision    recall  f1-score   support

           0       0.72      0.73      0.73      4995
           1       0.73      0.72      0.73      5005

    accuracy                           0.73     10000
   macro avg       0.73      0.73      0.73     10000
weighted avg       0.73      0.73      0.73     10000



The performance of this model is comparable to the SVC, Naive Bayes and Random Forest. Although it does slightly poor compared to the other models if we look at the precision scores. 

4. Compare all models together in terms of Precision, Recall and F1 score. Put all of
these numbers in a nicely formatted dataframe. Answer the following questions: Which
model performs the best? Why do you think this is? What do you think you can do to
improve performance?

In [14]:
import sklearn

Precision = []
Recall = []
F1Score = []

labels_nb = model_nb.predict(x_test.text.values)

Precision.append(sklearn.metrics.precision_score(y_test.values,labels_nb,pos_label=4))
Recall.append(sklearn.metrics.recall_score(y_test,labels_nb,pos_label=4))
F1Score.append( sklearn.metrics.f1_score(y_test,labels_nb, average='macro',pos_label=4))


labels_rf = model_rf.predict(x_test.text.values)
Precision.append(sklearn.metrics.precision_score(y_test,labels_rf,pos_label=4))
Recall.append(sklearn.metrics.recall_score(y_test,labels_rf,pos_label=4))
F1Score.append( sklearn.metrics.f1_score(y_test,labels_rf, average='macro',pos_label=4))



labels_svm = model_svm.predict(x_test.text.values)
Precision.append(sklearn.metrics.precision_score(y_test,labels_svm,pos_label=4))
Recall.append(sklearn.metrics.recall_score(y_test,labels_svm,pos_label=4))
F1Score.append(sklearn.metrics.f1_score(y_test,labels_svm, average='macro',pos_label=4))


Precision.append(sklearn.metrics.precision_score(y_test1,predicted_classes,pos_label=1))
Recall.append(sklearn.metrics.recall_score(y_test1,predicted_classes,pos_label=1))
F1Score.append(sklearn.metrics.f1_score(y_test1,predicted_classes, average='macro',pos_label=1))




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 4) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 4) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 4) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [15]:
print("Bayes")

print('Precision:', Precision[0])
print('Recall:', Recall[0])
print('F1 Score:',F1Score[0])

print("\nRandom Forest")

print('Precision:',Precision[1] )
print('Recall:',Recall[1] )
print('F1 Score:',F1Score[1] )

print("\nSVM")

print('Precision:',Precision[2] )
print('Recall:',Recall[2] )
print('F1 Score:',F1Score[2] )

print("\nLSTM")

print('Precision:',Precision[3] )
print('Recall:',Recall[3] )
print('F1 Score:',F1Score[3] )


Bayes
Precision: 0.7883574007220217
Recall: 0.6981018981018982
F1 Score: 0.7543212746008896

Random Forest
Precision: 0.7683686176836861
Recall: 0.7396603396603396
F1 Score: 0.7580241913989234

SVM
Precision: 0.774559686888454
Recall: 0.7908091908091908
F1 Score: 0.7800709143784266

LSTM
Precision: 0.7274371859296482
Recall: 0.7230769230769231
F1 Score: 0.7257989031956127


I think SVM does the best compared to all the other models. Although its Precision score is not the best which doesnt make it the best model to predict labels accurately, the recall score brings the whole score up for SVM meaning using this model we will make less mistakes compared to other models.

To improve performance we can remove the max_iter parameter in SVC and max_depth in Random Forest allowing the models to converge even further to get more accurate results. Also throwing more data at the model will increase the accuracy. Reading the documentation for the data tells us that it is auto labeled considering emoticons to determine the polarity. A data which is hand labelled might fare better as there is a lot more to the polarity of a text than the emoticons used.

5. Add to the comparison of #4 a the manually calculated precision, recall and F1 score
using VADER and their suggested defaults to categorize the test split tweets in positive
or negative. Answer the following questions: Is this approach as good as the previous
ones? Why do you think this is? (30 points)

In [16]:
import nltk
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()
vader_labels = []
for text in x_test.text.values:
  sentiment_scores = (sia.polarity_scores(text))
  if sentiment_scores['compound']>0:
    vader_labels.append(4)
  else:
    vader_labels.append(0)
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, vader_labels)
print("\nVader")
Precision1 = mat[0][0] / (mat[0][0] + mat[0][1])
print("Precision: ", str(Precision1))
Recall1 = mat[0][0] / (mat[0][0] + mat[1][0])
print("Recall: ",str(Recall1))
F_Measure1 = (2 * Precision1 * Recall1) / (Precision1 + Recall1)
print("F Measure: ",str(F_Measure1))

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

Vader
Precision:  0.6856856856856857
Recall:  0.6419868791002812
F Measure:  0.6631171345595354


Vader is a general pupose tool trained on social media text which makes use of lexical features (mainly words). Our dataset is auto labelled based on emoticons and not words. This might be the reason why vader did not perform as well as other models. 

Bonus (30 points): Try the following things to improve the LSTM model:

1) Use 90% training data, 10% testing

2) Remove stopwords from the tweets.

3) Remove all user mentions for the tweets (@something)

Compare all three new models in terms of their precision, recall and F1 score. Answer the
following questions: Did this change the results in any way? Why do you think so?

In [30]:
print(training_data)

         polarity  ...                                               text
1062755         4  ...  @writesfortea  Always happy to share the aweso...
436476          0  ...  Okay yay it's Sunday! Not  must feed my addict...
974124          4  ...  @jasons wow, I just saw this @ thingie, sorry ...
172330          0  ...  @monkeymoosh Happened to me the other day.  Wa...
548362          0  ...  my ears went up like a startled Dog's when i f...
...           ...  ...                                                ...
538585          0  ...  @RegularOlTy so pissed Ty. SO pissed. Dude, I ...
1522938         4  ...  @MissShimry  Maybe if you chill out it may go ...
1141533         4  ...  Trying to go to sleep before 8:30 for 6AM ride...
363651          0  ...       not the best way to start off summer, SICK. 
1285218         4  ...  Wax On. Wax Off .FIGHT! hehehe  http://punchmo...

[50000 rows x 6 columns]


In [32]:
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize 
from nltk.tokenize import TweetTokenizer

np.random.seed(2361)

traindata = {}
s_words = stopwords.words('english')
traindata['tweet_without_stopwords'] = training_data.text.apply(lambda x: ' '.join([word for word in x.split() if word not in (s_words)]))
tknzr = TweetTokenizer(strip_handles=True)
result = tknzr.tokenize(str(training_data.text))
traindata['tweet_without_usermention'] = traindata['tweet_without_stopwords'].apply(lambda x: ' '.join(tknzr.tokenize(x)))
x_train, x_test, y_train, y_test = train_test_split(traindata['tweet_without_usermention'], training_data.polarity,train_size=0.90)
y_train = y_train.to_frame()
y_test = y_test.to_frame()




In [34]:
np.random.seed(2361)
model_nb = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_nb.fit(x_train.values.ravel(), y_train.polarity.ravel())
labels2 = model_nb.predict(x_test.values)

In [ ]:
np.random.seed(2361)
y_train_values = y_train.polarity.ravel()
y_train_values = y_train_values.astype(np.int8)
model_rf = make_pipeline(TfidfVectorizer(), RandomForestClassifier(n_jobs=-1,warm_start=True,max_depth=1000))
model_rf.fit(x_train.values.ravel(), y_train_values)
labels3 = model_rf.predict(x_test.values)

In [38]:
np.random.seed(2361)
model_svm = make_pipeline(TfidfVectorizer(), SVC(max_iter=20000))
model_svm.fit(x_train.values.ravel(), y_train_values)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape=

In [40]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=280)
tokenizer.fit_on_texts(training_data.text.values) 
sequences_train = tokenizer.texts_to_sequences(x_train.values)
sequences_test = tokenizer.texts_to_sequences(x_test.values)

In [41]:
!pip install numpy==1.16.2

import tensorflow as tf 
import numpy as np
from tensorflow.keras.preprocessing import sequence
from numpy import array
from sklearn.metrics import classification_report


review_length = 280
vocab_size = 10000
x1_train = sequence.pad_sequences(sequences_train, maxlen = review_length)
x1_test = sequence.pad_sequences(sequences_test, maxlen = review_length)
model = tf.keras.models.Sequential()

model.add(
    tf.keras.layers.Embedding(
        input_dim = vocab_size, 
        output_dim = 32, 
        input_length = review_length 
    )
)
model.add(
    tf.keras.layers.Dropout(
        rate=0.25 
    )
)
model.add(
    tf.keras.layers.LSTM(
        units=32 
    )
)
model.add(
    tf.keras.layers.Dropout(
        rate=0.25 
    )
)
model.add(
    tf.keras.layers.Dense(
        units=1,
        activation='sigmoid'
    )
)
model.compile(
    loss=tf.keras.losses.binary_crossentropy, 
    optimizer=tf.keras.optimizers.Adam(), 
    metrics=['accuracy']) 
model.summary()
y_train1 = y_train
y_test1 = y_test
y_train1 = y_train1.replace(4,1)
y_test1 = y_test1.replace(4,1)
history = model.fit(x1_train, y_train1,batch_size=256,epochs=3,validation_split=0.2,verbose=1)

predicted_classes = model.predict_classes(x1_test)
print(predicted_classes)
classification_report = classification_report(y_test1, predicted_classes)
print(classification_report)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 280, 32)           320000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 280, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
141/141 [==============================] - 5s 24ms/step - loss: 0.6751 - accuracy: 0.5820 - val_loss: 0.

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[1]
 [0]
 [1]
 ...
 [0]
 [0]
 [0]]
              precision    recall  f1-score   support

           0       0.71      0.65      0.68      2471
           1       0.69      0.74      0.71      2529

    accuracy                           0.70      5000
   macro avg       0.70      0.70      0.70      5000
weighted avg       0.70      0.70      0.70      5000



In [44]:
import sklearn

Precision = []
Recall = []
F1Score = []

labels_nb = model_nb.predict(x_test.values)

Precision.append(sklearn.metrics.precision_score(y_test.values,labels_nb,pos_label=4))
Recall.append(sklearn.metrics.recall_score(y_test,labels_nb,pos_label=4))
F1Score.append( sklearn.metrics.f1_score(y_test,labels_nb, average='macro',pos_label=4))


labels_rf = model_rf.predict(x_test.values)
Precision.append(sklearn.metrics.precision_score(y_test,labels_rf,pos_label=4))
Recall.append(sklearn.metrics.recall_score(y_test,labels_rf,pos_label=4))
F1Score.append( sklearn.metrics.f1_score(y_test,labels_rf, average='macro',pos_label=4))



labels_svm = model_svm.predict(x_test.values)
Precision.append(sklearn.metrics.precision_score(y_test,labels_svm,pos_label=4))
Recall.append(sklearn.metrics.recall_score(y_test,labels_svm,pos_label=4))
F1Score.append(sklearn.metrics.f1_score(y_test,labels_svm, average='macro',pos_label=4))


Precision.append(sklearn.metrics.precision_score(y_test1,predicted_classes,pos_label=1))
Recall.append(sklearn.metrics.recall_score(y_test1,predicted_classes,pos_label=1))
F1Score.append(sklearn.metrics.f1_score(y_test1,predicted_classes, average='macro',pos_label=1))




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 4) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.4s finished
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 4) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 4) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single p

In [45]:
print("Bayes")

print('Precision:', Precision[0])
print('Recall:', Recall[0])
print('F1 Score:',F1Score[0])

print("\nRandom Forest")

print('Precision:',Precision[1] )
print('Recall:',Recall[1] )
print('F1 Score:',F1Score[1] )

print("\nSVM")

print('Precision:',Precision[2] )
print('Recall:',Recall[2] )
print('F1 Score:',F1Score[2] )

print("\nLSTM")

print('Precision:',Precision[3] )
print('Recall:',Recall[3] )
print('F1 Score:',F1Score[3] )


Bayes
Precision: 0.7616935483870968
Recall: 0.7469355476472914
F1 Score: 0.7537992023094156

Random Forest
Precision: 0.7461159530125048
Recall: 0.77856860419138
F1 Score: 0.7537219619461588

SVM
Precision: 0.7683114880493447
Recall: 0.7880585211546066
F1 Score: 0.7724623032874576

LSTM
Precision: 0.6874312935141077
Recall: 0.741795175958877
F1 Score: 0.6979958959527279


This did not make any significant changes in the results. SVM is still the best one among them and LSTM model still underperforms. We removed stop words and tweet mentions but we didnt remove the emoticons on which this dataset is classified. Emoticons are the lexicons that define the polarity in this case making removing stopo words and mentions not make a difference. 